# 使用Python制作简单的家用Bot

简单介绍了使用Python+IM制作家用bot的过程。

列举了一些家用Bot的实用功能和使用场景。

可以为有类似情况的朋友提供参考。

# 环境介绍

* 一个Debian based自组路由器
* 两个子网
* 一个NAS存储
* 一台电视+一个机顶盒

## 路由器设计

之所以使用Debian based system自己组路由器，主要是可以充分控制路由器上的各种功能。

包括子网拆分，IPv6，端口映射等等。

并且纯Linux的hook和API非常好做，因此可以充分打造各种自用工具。

## 网络结构

两个子网，指host和guest。

所有安全设备接入host，所有无线和非安全设备接入guest。

host可以直接访问guest，guest要访问host的资源需要拨VPN。

这样就杜绝了基于AP的各种攻击，例如无802.11x下的SSID欺骗。同时还能防止不授信设备和临时设备干扰运行。

缺点是D525不带AESNI指令集，同时各种VPN的Linux实现又普遍不支持多核，所以吞吐量并不高。

## NAS

NAS存储，指在host网络中放置的NAS设备。

同样是Debian系统。上面提供了samba/ssh/nextcloud三套访问界面。

## 电视机+机顶盒

除了标准的机顶盒播放外，还可以播放NAS上的文件。

# 通知功能

通知功能是bot最主要也是最好用的功能。借助通知，我们可以轻松的知道家里发生的一切。

## 通知基础原理

让所有应用和Bot对接通知功能无疑是非常繁琐，甚至近乎不可能的。最简单，也是最实用的办法，应该是所有应用和Bot之间使用一种标准协议连接。应用无需修改代码，Bot只需实现一次。幸运的是，我们其实有这种“标准通知协议”——只要发邮件就好。

我在Bot所在虚拟机上安装了exim4，在其他机器上使用sendemail或是其他SMTP库发送邮件。Bot会每隔一分钟检查一次邮件。如果有邮件，则按照规则收件，删除，并通知到IM上。其实理论来说，本机SMTP服务器可以做hook。但一分钟的延迟已经可以接受，所以我就不改了。

## 为什么不直接使用SMTP

理论上，既然已经使用SMTP，就可以将邮件发送到自己的邮箱，然后让邮件系统通知你(gmail/qq的邮件通知应用)。然而这种模式在实战中还是有些小小的问题。

1. 反垃圾邮件。如果要在家里把邮件发出去，有两个方法。
  1. 配置反向解析，SPF和DKIM。
  2. 使用某服务商的服务，通过帐号和密码寄出邮件。
2. 限额。有的时候在短时间内，SMTP会发出数十封邮件。

当然，使用WeChat的问题是，腾讯不欢迎WeChat bot。所以有时候bot不稳定。因此两种方法可以结合使用。

## 通知分级制

目前分为info和warn两个级别，有计划预留emerg级别。

所有级别的实现，均是在`/etc/aliases`中写入level: user。这样bot可以以user身份统一收信，发送方可以决定严重性级别。

bot收信后，按照当前过滤等级进行投递。如果大于等于当前级别，则将消息发送到IM上。如果小于当前级别，则不发送消息。

消息等级可以自动或手动控制。默认设定是9AM-10PM为info级，其余为warn级。

## DHCP通知

全网的DHCP使用dnsmasq发出。dnsmasq有个dhcp-script参数可以装hook。

`dhcp-script=/etc/dnsmasq.d/host_up.sh`

In [ ]:
%%bash
#!/bin/bash

if [ "$1" == "add" ]; then
  MAC=$2
  VENDER=$(python3 /home/shell/bin/macs.py "$MAC")
  sendemail\
    -s 192.168.1.1 -f dnsmasq@router.lan -t info@srv.lan\
    -u "dhcp offer to $4" -m "mac: $2 ($VENDER)\nip: $3 ($DNSMASQ_INTERFACE)"
fi

# logger $@

在host_up.sh脚本中，第二个参数为MAC，第三个参数为IP，第四个参数为名字。这里我使用我自己写的一个[pyoui](https://github.com/shell909090/pyoui)库，查找到对应Vender的名字。因此通知不但可以看到设备信息，还能看到Vender。以下是样例。

    from: dnsmasq@router
    to: info@bot
    dhcp offer to myiPad
    mac: 00:00:00:00:00:00 (Apple, Inc)
    ip: 192.168.1.2 (ens33)

## VPN接入/断开通知

由于有host/guest网络分别，所以经常有VPN拨入。此时可以用hook发送邮件，通知我们有人连入内网了。

`leftupdown=/etc/ipsec.d/updown`

In [ ]:
%%bash
#!/bin/bash

sendemail\
  -s 192.168.1.1 -f ipsec@router.lan -t warn@srv.lan\
  -u "ipsec $PLUTO_CONNECTION $PLUTO_VERB"\
  -m "user: $PLUTO_PEER_ID\nip: $PLUTO_PEER\nclientip: $PLUTO_PEER_CLIENT"

样例如下。

	from: ipsec@router
	to: warn@bot
	ipsec ipsec-in-ikev2 down-client
	user: 192.168.1.2
	ip: 192.168.1.2
	clientip: 10.1.1.1/32

## NAS访问通知

nextcloud支持SMTP发送通知邮件，因此我可以随时知道谁访问了什么文件。当然，现实中，我通知做了过滤，只监控了几个文件夹。

samba/ssh并不支持这种功能，所以非常遗憾。

## 各种监控通知

我部署了smokeping和munin。

* smokeping: 当某个远程设备无法ping通时告警。
* munin: 当某个设备指标异常时告警。

还可以接入各种监控系统。几乎所有监控系统都支持SMTP，这里就不一一详述。

## 如果路由器不是自己打造的

对SMTP的通知部分而言，不会有任何影响。但是由于DHCP钩子装不上去，因此需要向路由器做轮循。通过脚本抓取路由器管理界面的信息。这样也可以知道哪个设备连进来了。但缺点是工作繁琐，对路由器性能有损耗，通知时间长。

非自建的路由器一般没有两套子网，也没有VPN。所以这部分就不提了。

# 定时

定时功能非常简单，分为interval型和at型两种。interval隔一定时间执行一下，用于定时任务，例如检查邮件。有一个interval任务叫at，每隔60s执行一次。如果当前时间满足at的启动条件，则启动at任务。例如调整自动消息等级。

# 命令

## 命令的执行机制

由于bot不在路由器上，因此很多指令需要远程执行。我为bot生成了一个单独的ssh key，并添加到路由器上。原则上bot可以使用[paramiko](http://www.paramiko.org/)执行远程指令。但目前尚没有这么复杂的需求，都是subprocess+ssh执行的。

## level

手工指定消息级别。注意，手工指定后，如果不再一次设定手动制定级别为None，则手工指定的级别会持续强制覆盖自动级别。

## WOL

在路由器上执行[wake-on-line](https://en.wikipedia.org/wiki/Wake-on-LAN)，唤醒某个特定设备。

In [ ]:
@base.regcmd('wol')
@base.chkadm
def wol(bot, update):
    cmd = base.getcmd(update)
    if 'wol' in cfg and cmd in cfg['wol']:
        cmd = ['sudo', 'etherwake', cfg['wol'][cmd]]
    else:
        cmd = ['sudo', 'etherwake'] + cmd.split(' ')
    base.run_and_show(cmd, 10.0, bot, update)

两个用途。一方面，在远程我可以启动家里某个硬件设备（例如电脑，电视，播放器...）。这样可以设计所有设备可远程操控，又不用保持所有设备开机。另一方面，家里地方小，有些设备的开机按钮很难按到。这时候可以掏出手机来遥(zhuang)控(\*)。不过实际操控中，远程启动电视和播放器从来没用过。因为这两个设备不是当面开就没有意义...

## search

家里的NAS上有非常多的文件，因此做了一套文件索引管理系统。基本是两部分。

1. 能用mlocate的，使用mlocate管理。
2. 无法使用mlocate的，使用zgrep。

## 重启服务

虽说好的服务不需要重启，不过程序员的日常工作就是伺候不好的服务。

利用ssh+systemctl重启服务，简单且管用。

In [ ]:
restart_parser = argparse.ArgumentParser()
restart_parser.add_argument('-u', '--user', action='store_true')
restart_parser.add_argument('node', nargs='?', default='')
restart_parser.add_argument('srv')


@base.regcmd('restart')
@base.chkadm
def restart(bot, update):
    cmd = base.getcmd(update)
    args = restart_parser.parse_args(cmd.split(' '))
    if args.user:
        cmd = ['systemctl', '--user', 'restart', args.srv]
    else:
        cmd = ['sudo', 'systemctl', 'restart', args.srv]
    if args.node:
        cmd = ['ssh', args.node+'.lan',] + cmd
    base.run_and_show(cmd, 10, bot, update)

## NATS

NATS可以分析/搜索路由器上的conntrack表，列出家中所有使用外网的设备(不论tcp/udp)(IP/名称/MAC/Vender)。并且还可以详细了解某个具体IP的对外通讯情况(对端IP列表)。

如果NATS给出的信息还不够有提示性，我可以远程开启dnsmasq的`log-queries`。

	log-queries=extra
	log-facility=/var/log/dnsmasq.log

随后在dnsmasq的记录中grep搜索IP和query。可以获得这个IP的所有DNS请求。

In [ ]:
@base.regcmd('nats')
@base.chkadm
def nats(bot, update):
    cmd = ['ssh', 'router.lan', 'sudo', 'python3', 'bin/t.py'] + base.getcmd(update).split()
    base.run_and_show(cmd, 10, bot, update)

以下是输出样例。

    192.168.1.2 dsk 00:00:00:00:00:00 GIGA-BYTE TECHNOLOGY CO.,LTD
    192.168.1.3 pub 00:00:00:00:00:00 Xensource, Inc
    192.168.1.4 web 00:00:00:00:00:00 Xensource, Inc
    192.168.1.5 srv 00:00:00:00:00:00 Xensource, Inc
    192.168.2.2 meizu6t 00:00:00:00:00:00 Wingtech Group (HongKong）Limited
    192.168.2.3 pixel3 00:00:00:00:00:00 Google, Inc
    192.168.2.4 pixelxl 00:00:00:00:00:00 HTC Corporation
    192.168.2.5 Shell-MBPT-SH 00:00:00:00:00:00 Apple, Inc
    192.168.2.6 zhujialide-iPad 00:00:00:00:00:00 Apple, Inc

## 某些神秘的小功能

借助某些指令前缀，可以在bot中执行python代码，或者shell命令。这是一把双刃剑。

In [ ]:
@base.regcmd('reload')
@base.chkadm
def reload_self(bot, update):
    os.execvp('python', ['python', ]+sys.argv)

In [ ]:
@base.regcmd('py')
@base.chkadm
def run_python(bot, update):
    cmd = base.getcmd(update)
    exec(compile(cmd, '<bot python>', 'exec'))

In [ ]:
@base.regcmd('eval')
@base.chkadm
def run_eval(bot, update):
    cmd = base.getcmd(update)
    txt = str(eval(cmd))
    base.safe_send_msg(bot, update, txt)

In [ ]:
@base.regcmd('sh')
@base.chkadm
def run_sh(bot, update):
    cmd = base.getcmd(update)
    base.run_and_show(cmd, 10.0, bot, update)